In [2]:
import numpy as np
import pandas as pd
from sklearn import svm
import random
import os
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
import pickle

In [52]:
def training_function(SE,SE_drugs,all_drugs,features_dir,n_outer,MPP_dict,random_state=42):
    np.random.seed(random_state)
    for i in range(n_outer):
        print(i)
        n_pos=len(SE_drugs)
        n_neg=n_pos
        pos_SE_drugs = set(SE_drugs)
        eligible = [item for item in all_drugs if item not in pos_SE_drugs]
        pos_samples=SE_drugs
        try:
            neg_samples=random.sample(eligible,n_neg)#sampling negative samples to train on
        except:
            pos_samples=random.sample(SE_drugs,len(eligible))
            neg_samples=random.sample(eligible,len(eligible))
            n_pos=len(pos_samples)
            n_neg=len(neg_samples)
        
        for j in range(0,len(pos_samples)):
            #j'th drug will be left out
            #constructing test/leave out sets
            drug_pos_test=pos_samples[j]
            drug_neg_test=neg_samples[j]#randomly selecting leave out for negative
            drug_pos_test_df=pd.read_csv(f'{features_dir}fva_bounds_{drug_pos_test}.csv',header=None)
            drug_pos_test_pd=pd.concat([drug_pos_test_df[0],drug_pos_test_df[1]],axis=0,ignore_index=True)
            drug_neg_test_df=pd.read_csv(f'{features_dir}fva_bounds_{drug_neg_test}.csv',header=None)
            drug_neg_test_pd=pd.concat([drug_neg_test_df[0],drug_neg_test_df[1]],axis=0,ignore_index=True)
            X_test=np.array([drug_pos_test_pd,drug_neg_test_pd])
            Y_test=np.array([1,0])
            #constructing the training set
            X_train=np.zeros((2*(n_pos),4732))
            Y_train=np.zeros((2*(n_pos),1))
            for k_pos in range(0,n_pos):
                
                drug_pos_train_df=pd.read_csv(f'{features_dir}fva_bounds_{pos_samples[k_pos]}.csv',header=None)
                drug_pos_train_pd=pd.concat([drug_pos_train_df[0],drug_pos_train_df[1]],axis=0,ignore_index=True)
                #print(f'changing {k_pos} elements')
                X_train[k_pos]=drug_pos_train_pd
                Y_train[k_pos]=1

            for k_neg in range(0,n_pos):
                
                drug_neg_train_df=pd.read_csv(f'{features_dir}fva_bounds_{neg_samples[k_neg]}.csv',header=None)
                drug_neg_train_pd=pd.concat([drug_neg_train_df[0],drug_neg_train_df[1]],axis=0,ignore_index=True)
                X_train[n_pos + k_neg]=drug_neg_train_pd
                #print(f'changed {(n_pos-1) + k_neg} element')
                Y_train[n_pos + k_neg]=0
            #training set has been constructed
            X_train_master=X_train
            Y_train_master=Y_train
            
    #training and saving MPP to a dict file
    clf = RandomForestClassifier(n_estimators=1000, random_state=42)
    clf.fit(X_train_master,np.ravel(Y_train_master))
    
    non_zero_values = [x for x in clf.feature_importances_ if x >=0.01]
    print(len(clf.feature_importances_))
    temp_imp_dict={}
    for i in range(0,len(clf.feature_importances_)):

        if i >2365:
            if all_rxns[0][i-2366] not in exchange_rxns[0].values:
                continue
            temp_imp_dict[all_rxns[0][i-2366]]=temp_imp_dict[all_rxns[0][i-2366]]+clf.feature_importances_[i]
            
        else:
            if all_rxns[0][i] not in exchange_rxns[0].values:
                continue
            temp_imp_dict[all_rxns[0][i]]=clf.feature_importances_[i]
            
    feature_imp[SE]=temp_imp_dict
    print(f'Trained MPP for {SE}')
    
    
    return 0

In [ ]:
fva_bounds_norm_dir='C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/FVA_Bounds_norm3/'
df_all_drugs=pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drug_gene_targets_meta_1.csv")
all_drugs=df_all_drugs['dg_id']
all_rxns=pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/GSMMs/all_reactions.csv",header=None)
exchange_rxns=pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/GSMMs/exchange_reactions.csv",header=None)
#print(len(all_drugs))
df_SE=pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/side_effects_vs_causative_drugs_meta.csv")
#AMPP Training only
MPP_dict={}
fpr_dict={}
tpr_dict={}
feature_imp={}

n_outer=1
for k in range(0,1):
    for se in range(0,len(df_SE['concept_id'])):
        
        '''if se>2:
            continue
        if (df_SE['concept_id'][se]!='C0000731'):
            continue'''
        SE_drugs=df_SE['dg_id'][se].split(',')
        for i in range(0,len(SE_drugs)):
            SE_drugs[i]=SE_drugs[i].lstrip()
        print(f'starting training {df_SE['concept_id'][se]}')
        
        n=training_function(df_SE['concept_id'][se],SE_drugs,all_drugs,fva_bounds_norm_dir,n_outer,MPP_dict)

with open('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/AMPP/AMPP_rf.pkl', 'wb') as file:
    pickle.dump(feature_imp, file)


In [4]:
with open('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/AMPP/AMPP_rf.pkl', 'rb') as file:
    loaded_dict = pickle.load(file)
loaded_dict
for i in loaded_dict.keys():
    print(i)
    top_10 = sorted(loaded_dict[i].items(), key=lambda x: x[1], reverse=True)[:3]

# Print key and value
    for k, v in top_10:
        print(f"{k}: {v}")

C0000731
EX_h_e: 0.004031658262750701
EX_3hanthrn_e: 0.003862043482802345
EX_dxtrn_e: 0.0035975301995796283
C0000737
EX_pi_e: 0.0053108440214982575
EX_adn_e: 0.005172128140215798
EX_amp_e: 0.004793748586738317
C0000833
EX_co2_e: 0.0037361878256361135
EX_asn__L_e: 0.0034855228434884766
EX_pi_e: 0.003277388191351769
C0001824
EX_no_e: 0.008657026763026345
EX_prostgd2_e: 0.0035112725153404454
EX_arggluglu_e: 0.003391466481356663
C0001925
EX_cit_e: 0.010102151590559192
EX_csn_e: 0.005948269681103362
EX_asn__L_e: 0.0053151982936898625
C0002170
EX_csn_e: 0.005298527911298465
EX_5fthf_e: 0.004266223569798485
EX_lac__L_e: 0.004003410924717584
C0002622
EX_5mthf_e: 0.004010813894068436
EX_dxtrn_e: 0.003380115864436067
EX_oxa_e: 0.003268033745625158
C0002792
EX_no_e: 0.007973633239115865
EX_cit_e: 0.0028031716805602493
EX_pi_e: 0.0026702799324208612
C0002871
EX_pi_e: 0.0060812491283570415
EX_amp_e: 0.004938449512458828
EX_csn_e: 0.0048656875535227585
C0002874
EX_dxtrn_e: 0.006720812142200087
EX_cs

In [ ]:
# Sample dictionary

# Get top 10 items by value
top_10 = sorted(feature_imp['C0000731'].items(), key=lambda x: x[1], reverse=True)[:3]

# Print key and value
for k, v in top_10:
    print(f"{k}: {v}")
